# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Model 
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint 
import joblib
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#ANCLA:una vez que ya fue leida la data enviar un mensaje de realizado
#ANCLA: enviar una salida de datos del dataframe para que pueda visualizarse
#getting data: the dataset was uploaded to Azure Environment but can be accesible through an online link for example
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
pdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
#ANCLA: hacer un check en caso que el experimento ya exista, enviar un mensaje que fue creado o que ya existe segun sea el caso
#setting up workspace and dataset
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'exp-p03-ht'

experiment=Experiment(ws, experiment_name)

In [4]:
# setting up IaaS where the experiment will run on it
ccname = "cc-p03-aml"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists or is in use!, choose a different name")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
#ANCLA: enviar un mensaje que las configs fueron seteadas
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=ccluster,
        training_data=dshomic2018,
        label_column_name='sexo',
        n_cross_validations=5,
        **automl_settings
)

In [6]:
# TODO: Submit your experiment
exprun = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(exprun).show()
exprun.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of 

        60   StandardScalerWrapper XGBoostClassifier        0:00:42       0.8790    0.8811
        61   MaxAbsScaler LightGBM                          0:00:34       0.8790    0.8811
        62   SparseNormalizer XGBoostClassifier             0:00:41       0.8787    0.8811
        63   StandardScalerWrapper LightGBM                 0:00:40       0.8790    0.8811
        64   SparseNormalizer XGBoostClassifier             0:00:37       0.8808    0.8811
        65   TruncatedSVDWrapper XGBoostClassifier          0:00:42       0.8790    0.8811
        66   StandardScalerWrapper LightGBM                 0:00:39       0.8790    0.8811
        67   SparseNormalizer LightGBM                      0:00:42       0.8790    0.8811
        68   MaxAbsScaler LightGBM                          0:00:35       0.8790    0.8811
        69   SparseNormalizer XGBoostClassifier             0:00:44       0.8790    0.8811
        71   MaxAbsScaler GradientBoosting                  0:00:37       0.8790    0.8811

{'runId': 'AutoML_7c0770c3-8a83-442e-b93f-94199bbda58a',
 'target': 'cc-p03-aml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T11:08:45.592356Z',
 'endTimeUtc': '2021-01-17T11:39:51.255327Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cc-p03-aml',
  'AMLSettingsJsonString': '{"path":null,"name":"exp-p03-ht","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-135033","workspace_name":"quick-starts-ws-135033","region":"southcentralus","compute_target":"cc-p03-aml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featuri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# obtaining best run and fitted model
brmodel, fmodel = exprun.get_output()

# Print the best run
print(brmodel)

# Get all metrics of the best run model
brmetrics = brmodel.get_metrics()

# Print all metrics of the best run model
for metric_name in brmetrics:
    metric = brmetrics[metric_name]
    print(metric_name, metric)

Run(Experiment: exp-p03-ht,
Id: AutoML_7c0770c3-8a83-442e-b93f-94199bbda58a_85,
Type: azureml.scriptrun,
Status: Completed)
AUC_micro 0.9039128018545052
recall_score_weighted 0.8813497534747785
recall_score_macro 0.5132251435892704
precision_score_macro 0.8575440907689178
average_precision_score_micro 0.8862275012606216
recall_score_micro 0.8813497534747785
matthews_correlation 0.13428777196960962
AUC_weighted 0.6176009141515569
average_precision_score_weighted 0.8288336629770946
precision_score_weighted 0.8755610197492085
weighted_accuracy 0.980851718721591
f1_score_macro 0.494832026757759
f1_score_weighted 0.8296767613614848
log_loss 0.35873146548801094
precision_score_micro 0.8813497534747785
average_precision_score_macro 0.5677144887258861
AUC_macro 0.6176009141515568
f1_score_micro 0.8813497534747785
accuracy 0.8813497534747785
balanced_accuracy 0.5132251435892704
norm_macro_recall 0.02645028717854081
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_7c0770c3-8a83-442e-b93

In [22]:
#TODO: Save the best model
autobmlp3ht = brmodel.register_model(model_path='.', model_name='p3automl_ht',
                        tags={'Method of execution':'Auto ML'},
                        properties={'Accuracy': brmetrics['accuracy']})

print(autobmlp3ht)

Model(workspace=Workspace.create(name='quick-starts-ws-135033', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-135033'), name=p3automl_ht, id=p3automl_ht:1, version=1, tags={'Method of execution': 'Auto ML'}, properties={'Accuracy': '0.8813497534747785'})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [25]:
# Download scoring file 
brmodel.download_file('outputs/scoring_file_v_1_0_0.py', 'scoring.py')

# Download environment file
brmodel.download_file('outputs/conda_env_v_1_0_0.yml', 'enviro.yml')

In [28]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

infcfg = InferenceConfig(entry_script='scoring.py',
                                    environment=brmodel.get_environment())

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wsp03ht01", [autobmlp3ht], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 3a41ff9d-0194-4051-8a54-82f6a054228c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: wsp03ht01. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more i

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 3a41ff9d-0194-4051-8a54-82f6a054228c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: wsp03ht01. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a locally. Please refer to https://aka.ms/debugimage#service-launch-fails

In [ ]:
import json

#Importing the data set for testing 
tsthomic2018 = pdhomic2018.sample(5) 
lblhomic2018 = tsthomic2018.pop('sexo')

tstdatahomic2018 = json.dumps({'pdhomic2018': tsthomic2018.to_dict(orient='records')})

print(tstdatahomic2018)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [ ]:
# Print results from the inference
print(response.text)

In [ ]:
# Print original labels
print(lblhomic2018)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(webservice.get_logs())

In [ ]:
webservice.delete()